# ML Dataset EDA - A32S Station

Exploratory Data Analysis for headway prediction dataset

- **Station:** West 4th St-Washington Square (A32S - Southbound)
- **Tracks:** A1 (local), A3 (express)
- **Date Range:** July 18, 2025 - January 19, 2026
- **Features:** headway, time_of_day, day_of_week, route_id

---

**Note:** This notebook uses [eda_utils.py](eda_utils.py) for clean visualization functions. All analysis code has been abstracted to keep this notebook focused on insights and charts.

In [ ]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
from eda_utils import *

# Initialize BigQuery client
client = bigquery.Client()

In [ ]:
# Load data from BigQuery
query = """
SELECT * 
FROM `headway_prediction.ml`
ORDER BY track, arrival_time
"""
df = client.query(query).to_dataframe()

# Display dataset overview
print_dataset_overview(df)

In [ ]:
# Separate tracks for analysis
df_a1 = df[df.track == 'A1'].copy()
df_a3 = df[df.track == 'A3'].copy()

# Print A1 track statistics
print_track_statistics(df_a1, track_name="A1")

In [ ]:
# Check for missing values and basic statistics
print_missing_values(df_a1, track_name="A1")

In [ ]:
# A1 Headway distribution and outlier analysis
outlier_stats = plot_headway_distribution(df_a1, track_name="A1")

In [ ]:
# Check log transformation effectiveness
df_a1_clean = plot_log_transformation(df_a1, track_name="A1")

## Log Transformation Results

**Raw skewness:** 143.920 - Extremely right-skewed  
**Log skewness:** 0.834 - Nearly normal distribution (close to 0)  
**Impact:** ~170x reduction in skewness!

Log transformation is highly effective for A1 local track.

In [ ]:
# Temporal patterns: hourly headway and frequency
hourly_stats, peak_hours, offpeak_hours = plot_temporal_patterns(df_a1_clean, track_name="A1")

## Temporal Pattern Analysis - Key Findings

### Service Patterns
- **Peak vs Off-peak:** 3.87 min vs 5.71 min - ~32% shorter headway during rush hours (more frequent service)
- **Overnight variability spike (hour 5):** Huge standard deviation - service gaps causing outliers
- **Consistent daytime frequency:** Hours 6-20 show steady train arrivals (~2,500-2,900 per hour)
- **Reduced overnight service:** Hours 0-5 and 22-23 show lower frequency

### Model Implications
- `hour_of_day` is a strong predictor
- May need special handling for overnight periods
- Time-based patterns are clear and learnable

In [ ]:
# Heatmap: Hour of Day × Day of Week
heatmap_data = plot_heatmap(df_a1_clean, track_name="A1")

## Heatmap Insights - Hour × Day of Week

### Key Patterns
- **Monday hour 5:** 39.9 min - Major outlier (dark red) - likely a service transition gap
- **Peak hours (7-9 AM, 16-19 PM):** Consistently 3-4 min (dark green) across all days
- **Overnight (0-4, 22-23):** 5-12 min headways (lighter green/yellow)
- **Weekends vs Weekdays:** Subtle but visible - weekends slightly longer overall
- **Hour 6:** Clear transition from overnight to peak service (drops from ~10 min to ~5 min)

In [ ]:
# Day of week analysis
dow_stats, weekday, weekend = plot_day_of_week_analysis(df_a1_clean, track_name="A1")

In [ ]:
# Comprehensive weekday vs weekend comparison
plot_weekday_weekend_comparison(weekday, weekend, track_name="A1")

# Express Track Dataset

In [ ]:
# A3 Express Track - Basic statistics
df_a3_clean = df_a3[df_a3['headway'].notna()].copy()
df_a3_clean['log_headway'] = np.log(df_a3_clean['headway'] + 1)

print_track_statistics(df_a3, track_name="A3 (Express)")
print("\n" + "=" * 60)
print_missing_values(df_a3, track_name="A3")

In [ ]:
# A3 vs A1 Track comparison
comparison_stats = plot_track_comparison(df_a1_clean, df_a3_clean)

## A3 Express Track Analysis

### Skewness Comparison
- **A3 Raw:** 21.872 vs **A1:** 143.920 → A3 is ~6.5x less skewed (more regular service)
- **A3 Log:** 1.808 vs **A1:** 0.834 → Log transform helps A3 less (still somewhat right-skewed)

### Outlier Analysis
- **A3 threshold:** 18.95 min vs **A1:** 11.92 min → Express has longer typical headways
- **A3 outliers:** 3.35% vs **A1:** 6.12% → Express has fewer extreme outliers

### Key Insights
- **Express track is more predictable** - lower raw skewness suggests more consistent service intervals
- **Longer but more regular headways** - express trains run less frequently but more reliably
- **Fewer service disruptions** - lower outlier percentage
- **Distribution comparison** - A3 (red) is shifted right with higher median/mean headway

This makes sense - express trains have fewer stops, so they can maintain more consistent schedules. The log transformation may be less necessary for A3 than A1.

In [ ]:
# A3 Heatmap and weekday/weekend comparison
heatmap_data_a3 = plot_heatmap(df_a3_clean, track_name="A3 (Express)")

# Calculate weekday vs weekend statistics for A3
weekday_a3 = df_a3_clean[df_a3_clean['day_of_week'].isin([2, 3, 4, 5, 6])]
weekend_a3 = df_a3_clean[df_a3_clean['day_of_week'].isin([1, 7])]

print("\nA3 Track - Weekday vs Weekend:")
print(f"Weekday mean headway: {weekday_a3['headway'].mean():.2f} min")
print(f"Weekend mean headway: {weekend_a3['headway'].mean():.2f} min")
print(f"Difference: {((weekend_a3['headway'].mean() - weekday_a3['headway'].mean()) / weekday_a3['headway'].mean() * 100):.1f}% longer on weekends")

## A3 Express Track - Critical Findings

### 1. Extreme Overnight Outliers
- **Sunday hour 2:** 194.9 min (over 3 hours!)
- **Sunday hour 7:** 167.7 min
- **Hour 5 across week:** 190.9, 163.1, 122.8, 155.4, 160.2 → Major service gaps

### 2. Weekend Impact is MASSIVE
- **69.5% longer headways on weekends** (vs A1's 28%)
- **Weekday:** 10.08 min vs **Weekend:** 17.09 min
- Express service is highly schedule-dependent

### 3. Track Comparison (Weekday)
- **A3 Express:** 10.08 min
- **A1 Local:** 4.73 min
- Express runs ~2x less frequently than local

### 4. Service Variability
- Many yellow/orange cells showing 20-60+ min gaps
- Less predictable than local track
- Overnight service especially erratic

### Model Implications
- **A3 may need separate model** - very different patterns from A1
- **`day_of_week` is CRITICAL** predictor for A3 (weekend flag essential)
- **Overnight hours (0-6)** need special handling or filtering

### Key Decisions Needed
1. Train separate models per track vs combined model with track feature
2. Filter extreme outliers differently per track
3. Handle overnight service specially

**Status:** EDA Complete ✅

# EDA Summary & Recommendations

## Dataset Overview
- **Total Records:** 75,390 arrivals (July 18, 2025 - Jan 19, 2026)
- **Track A1 (Local):** 52,085 examples | **Track A3 (Express):** 23,305 examples
- **Station:** A32S (West 4th St - Southbound)
- **Routes:** Primarily E (56%), C (33%), A (10%) on local; A-dominant on express

---

## Key Findings

### 1. Track Characteristics - Distinct Patterns

| Metric | A1 (Local) | A3 (Express) | Insight |
|--------|-----------|--------------|---------|
| **Mean Headway** | 5.12 min | 11.42 min | Express runs 2x less frequently |
| **Median Headway** | 3.58 min | 7.42 min | Confirms frequency difference |
| **Raw Skewness** | 143.9 | 21.9 | Local has extreme outliers |
| **Outliers** | 6.12% | 3.35% | Express more predictable |
| **Weekend Impact** | +28% | +69.5% | Express weekend service drastically reduced |

**Recommendation:** Train **separate models per track** - patterns are too different to combine effectively.

---

### 2. Feature Effectiveness

#### ✅ Strong Predictors
- **`hour_of_day`:** Clear peak (7-9 AM, 5-7 PM) vs off-peak patterns
  - Peak: 3.87 min | Off-peak: 5.71 min (32% difference)
- **`day_of_week`:** Weekday vs weekend distinction critical (especially for A3)
- **`route_id`:** Composite headway captured, multiple routes per track

#### ⚠️ Temporal Features
- **`time_of_day_seconds`:** Cyclical encoding (sin/cos) will be essential
- **Overnight hours (0-6 AM):** Extreme variability - consider filtering or separate treatment

---

### 3. Data Quality

| Issue | Finding | Action |
|-------|---------|--------|
| **Missing Values** | <0.01% null headways | Negligible - drop nulls |
| **Outliers** | 6.12% (A1), 3.35% (A3) above IQR threshold | Consider capping at 120 min or use robust loss |
| **Service Gaps** | Mon hour 5: 39.9 min (A1), 194.9 min (A3 Sun) | Filter overnight hours or flag as special case |
| **Route Distribution** | ~1% "other" routes (F, M, B, D) | Acceptable - include for robustness |

---

### 4. Preprocessing Requirements

#### Log Transformation
- **A1:** Highly effective (skew: 143.9 → 0.834) ✅
- **A3:** Less effective (skew: 21.9 → 1.808) - consider MinMax scaling instead

#### Temporal Encoding
- **Time of day:** `sin(2π·t/86400)`, `cos(2π·t/86400)`
- **Day of week:** Binary weekend flag OR cyclical encoding
- **Route ID:** One-hot encoding (A, C, E) - 3 dimensions

#### Sequence Creation
- **Lookback window:** 10-15 previous events (based on distribution analysis)
- **Gap handling:** Break sequences at >120 min gaps (overnight service breaks)
- **Per-track processing:** Separate sequences for A1 and A3

---

### 5. Model Architecture Recommendations

#### Option A: Separate Models (RECOMMENDED)

**Model A1 (Local):**
- Input: `(batch, 15, 7)` - 15 events, 7 features
- Architecture: Stacked GRU or LSTM
- Loss: Huber (robust to outliers)

**Model A3 (Express):**
- Input: `(batch, 15, 7)`
- Same architecture, different weights
- Loss: MSE (fewer outliers, more regular)

**Rationale:** 
- Different service patterns (2x frequency difference)
- Weekend impact differs dramatically (28% vs 69%)
- Allows per-track optimization

#### Option B: Single Model with Track Feature
- Add `track` as binary feature (0=A1, 1=A3)
- May require more capacity to learn distinct patterns
- Risk: suboptimal performance on both tracks

---

### 6. Training Strategy

#### Data Splits (Temporal)
- **Train:** July 18 - Nov 26, 2025 (~60%)
- **Validation:** Nov 27 - Dec 27, 2025 (~20%)
- **Test:** Dec 28, 2025 - Jan 19, 2026 (~20%)

Use `timeseries_dataset_from_array()` with index ranges for clean temporal split.

#### Handling Imbalance
- A1: 52k examples | A3: 23k examples
- A3 model may need:
  - Data augmentation (jittering, sliding window overlap)
  - Transfer learning from A1 model
  - Regularization to prevent overfitting

## Next Steps

### Phase 1: Data Pipeline (Week 1)
1. ✅ SQL transformation complete - `ml` dataset ready
2. ⬜ Create preprocessing module (`src/data/preprocessing.py`)
   - Log transform, temporal encoding, one-hot encoding
   - Sequence generation with gap detection
   - Train/val/test split logic
3. ⬜ Build dataset loader (`src/data/dataset.py`)
   - TensorFlow `timeseries_dataset_from_array` wrapper
   - Separate loaders for A1 and A3

### Phase 2: Model Development (Week 1-2)
1. ⬜ Implement Stacked GRU architecture (`src/models/model.py`)
2. ⬜ Configure training loop (`src/training/trainer.py`)
   - Vertex AI Experiments integration
   - TensorBoard logging
   - Early stopping, checkpointing
3. ⬜ Define metrics (`src/metrics.py`)
   - MAE, RMSE for headway prediction
   - Per-hour, per-day performance breakdown

### Phase 3: Training & Evaluation (Week 2)
1. ⬜ Train A1 model (local track)
2. ⬜ Train A3 model (express track)
3. ⬜ Run evaluator on test set (`src/evaluator.py`)
4. ⬜ Generate performance report with TensorBoard visualizations

### Phase 4: Production Deployment (Week 3)
1. ⬜ Register models to Vertex AI Model Registry
2. ⬜ Deploy prediction endpoints
3. ⬜ Integration with real-time GTFS pipeline

---

## Open Questions for Next Session
1. Should we filter overnight hours (0-6 AM) or handle them separately?
2. Do we need multi-output model (headway + train type) or just headway?
3. What's the target inference latency for production?
4. Should we implement ensemble (A1 + A3 models combined)?

---

**Status:** ✅ EDA Complete | **Next:** Preprocessing Pipeline Development